In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import funcoes
import itertools
from sklearn.metrics import accuracy_score
from sklearn import svm

sns.set_style('darkgrid')
%matplotlib inline

In [3]:
df = pd.read_csv('conhecimento.csv')
df['dia'] = df.id_cena.str[-3:].astype(int)

id_cenas = [
    'LS82260702017256', # 226_070
    'LS82210702017269', # 221_070
    'LS82210672017285', # 221_067
    'LS82210742018128', # 221_074
]
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']

In [4]:
def gera_teste_modelo(id_cena):
    
    teste_original = df[df.id_cena==id_cena].copy().reset_index(drop=True)
    teste = teste_original.copy()
    teste['verifica'] = 0

    # Modelo
    modelo_inteiro = df[~(df.id_cena==id_cena)].copy()

    modelo_inteiro_1 = modelo_inteiro[modelo_inteiro.verifica==1].copy()
    modelo_inteiro_3_full = modelo_inteiro[modelo_inteiro.verifica==3].copy()

    id3 = np.random.choice(modelo_inteiro_3_full.id, len(modelo_inteiro_1), replace=False) 
    modelo_inteiro_3 = modelo_inteiro_3_full[modelo_inteiro_3_full.id.isin(id3)]

    modelo = pd.concat([modelo_inteiro_1, modelo_inteiro_3]).reset_index(drop=True)

    return teste, teste_original, modelo
def _resultado(teste, teste_original, pred, verdade,verifica=1):
    r = pd.merge(teste[['id', pred]], teste_original[['id', verdade]], on='id')
    r['v'] = r[pred].astype(str) + r[verdade].astype(str)
    
    try:
        x = len(r[(r[verdade]==verifica) & (r[pred] == verifica)])
        v = len(r[r[verdade]==verifica])
        p = int((x*100)/v)
    except:
        return [0, 0, 0]
    return [x, v, p]

In [5]:
def class_rf(dado, modelo, colunas):
    classificador = RandomForestClassifier(n_jobs=8, n_estimators=100, max_features=len(melhores_colunas), random_state=0, max_depth=3, min_samples_split=5)
    classificador.fit(modelo[colunas], modelo.verifica)
    return classificador.predict(dado[colunas])

def class_nn(dado, modelo, colunas):
    classificador = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.00010, max_iter=200)
    classificador.fit(modelo[colunas], modelo.verifica)
    return classificador.predict(dado[colunas])

def class_svm(dado, modelo, colunas):
    classificador = svm.SVC(kernel='rbf',C=1.0, gamma= 0.1)
#     classificador = svm.SVC(gamma='scale')
    classificador.fit(modelo[colunas], modelo.verifica)
    return classificador.predict(dado[colunas])

def class_knn(dado, colunas):
    dado = dado.copy()
    dados = dado[colunas].values
    classificador = KMeans(n_clusters=2, init='k-means++', n_init=10, max_iter=300, n_jobs=8)
    classificador.fit(dados)
    r = classificador.predict(dados)
    
    dado['knn'] = r
#     dado['color'] = dado.knn.apply(lambda x: 'r' if x else 'b')
#     dado.plot.scatter(x='medianb5',y='nbrl', figsize=(15, 5),s=80,c=dado.color)
    c0 = dado[dado['knn']==0].medianb5.max()
    c1 = dado[dado['knn']==1].medianb5.max()
    melhor = 0
    if c1 < c0:
        melhor = 1
    return np.where(r==melhor, 1, 0)

def ver_resultado(teste, teste_original, nome_classe):
    y_pred = teste.sort_values('id').reset_index(drop=True)
    y_true = teste_original.sort_values('id').reset_index(drop=True)
    r = _resultado(y_pred, y_true, nome_classe, 'verifica')
    r.append(nome_classe)
    return r

In [6]:
melhores_colunas = ['ndvi', 'nbrl', 'dif_ndvi', 'dif_dnbrl', 'medianb2', 'medianb3', 'medianb4', 'medianb5', 'medianb6', 'medianb7']

In [7]:
import datetime as dt

In [13]:
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo(id_cena)
    
    nome_classe = 'rf'
    teste[nome_classe] = class_rf(teste, modelo, melhores_colunas)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1.5 if x == 1 else 0) # coloca peso 1.5
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
    
    nome_classe = 'nn'
    teste[nome_classe] = class_nn(teste, modelo, melhores_colunas)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1.5 if x == 1 else 0) # coloca peso 1.5
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
    nome_classe = 'knn'
    teste[nome_classe] = class_knn(teste, melhores_colunas)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1 if x == 1 else 0) # coloca peso 1
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
    nome_classe = 'nota'
    teste['avaliacao'] = teste.rf + teste.nn + teste.knn
    teste[nome_classe] = teste.avaliacao.apply(lambda x: 1 if x > 3 else 3)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
#     print(dt.datetime.now(), 'inicio')
#     nome_classe = 'svm'
#     teste[nome_classe] = class_svm(teste, modelo, melhores_colunas)
#     teste[nome_classe] = teste[nome_classe].apply(lambda x: 1 if x == 1 else 0) # coloca peso 1
#     resultado = ver_resultado(teste, teste_original, nome_classe)
#     resultado.append(id_cena)
#     print(resultado)
#     sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
#     print(dt.datetime.now(), 'fim')

#     break

[805, 805, 100, 'rf', 'LS82260702017256']
[802, 805, 99, 'nn', 'LS82260702017256']
[797, 805, 99, 'knn', 'LS82260702017256']
[794, 805, 98, 'nota', 'LS82260702017256']
[547, 599, 91, 'rf', 'LS82210702017269']
[530, 599, 88, 'nn', 'LS82210702017269']
[548, 599, 91, 'knn', 'LS82210702017269']
[527, 599, 87, 'nota', 'LS82210702017269']
[1089, 1102, 98, 'rf', 'LS82210672017285']
[1086, 1102, 98, 'nn', 'LS82210672017285']
[1102, 1102, 100, 'knn', 'LS82210672017285']
[1083, 1102, 98, 'nota', 'LS82210672017285']
[920, 1072, 85, 'rf', 'LS82210742018128']
[994, 1072, 92, 'nn', 'LS82210742018128']
[914, 1072, 85, 'knn', 'LS82210742018128']
[786, 1072, 73, 'nota', 'LS82210742018128']


In [ ]:
a=1

In [ ]:
teste, teste_original, modelo = gera_teste_modelo(id_cena)
teste[nome_classe] = class_svm(teste, modelo, melhores_colunas)

In [8]:
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo(id_cena)
    
    nome_classe = 'rf'
    teste[nome_classe] = class_rf(teste, modelo, melhores_colunas)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1.5 if x == 1 else 0) # coloca peso 1.5
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
    
    nome_classe = 'nn'
    teste[nome_classe] = class_nn(teste, modelo, melhores_colunas)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1.5 if x == 1 else 0) # coloca peso 1.5
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
    nome_classe = 'knn'
    teste[nome_classe] = class_knn(teste, melhores_colunas)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1 if x == 1 else 0) # coloca peso 1
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    
    nome_classe = 'svm'
    teste[nome_classe] = class_svm(teste, modelo, melhores_colunas)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1 if x == 1 else 0) # coloca peso 1
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)

    nome_classe = 'nota'
    teste['avaliacao'] = teste.rf + teste.nn + teste.knn + teste.svm
    teste[nome_classe] = teste.avaliacao.apply(lambda x: 1 if x > 2 else 3)
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)

#     break

[805, 805, 100, 'rf', 'LS82260702017256']
[797, 805, 99, 'nn', 'LS82260702017256']
[797, 805, 99, 'knn', 'LS82260702017256']
[805, 805, 100, 'svm', 'LS82260702017256']
[805, 805, 100, 'nota', 'LS82260702017256']
[547, 599, 91, 'rf', 'LS82210702017269']
[532, 599, 88, 'nn', 'LS82210702017269']
[545, 599, 90, 'knn', 'LS82210702017269']
[528, 599, 88, 'svm', 'LS82210702017269']
[543, 599, 90, 'nota', 'LS82210702017269']
[1088, 1102, 98, 'rf', 'LS82210672017285']
[1085, 1102, 98, 'nn', 'LS82210672017285']
[1102, 1102, 100, 'knn', 'LS82210672017285']
[1081, 1102, 98, 'svm', 'LS82210672017285']
[1092, 1102, 99, 'nota', 'LS82210672017285']
[929, 1072, 86, 'rf', 'LS82210742018128']
[973, 1072, 90, 'nn', 'LS82210742018128']
[914, 1072, 85, 'knn', 'LS82210742018128']
[959, 1072, 89, 'svm', 'LS82210742018128']
[1008, 1072, 94, 'nota', 'LS82210742018128']


In [9]:
sumario

,previsao,verdade,porcentagem,id_cena,caso
0,805,805,100,rf,LS82260702017256
1,797,805,99,nn,LS82260702017256
2,797,805,99,knn,LS82260702017256
3,805,805,100,svm,LS82260702017256
4,805,805,100,nota,LS82260702017256
5,547,599,91,rf,LS82210702017269
6,532,599,88,nn,LS82210702017269
7,545,599,90,knn,LS82210702017269
8,528,599,88,svm,LS82210702017269
9,543,599,90,nota,LS82210702017269


In [10]:
sumario.to_csv('resultado_4classes.csv', index=False)

In [14]:
sumario

,previsao,verdade,porcentagem,id_cena,caso
0,805,805,100,rf,LS82260702017256
1,802,805,99,nn,LS82260702017256
2,797,805,99,knn,LS82260702017256
3,794,805,98,nota,LS82260702017256
4,547,599,91,rf,LS82210702017269
5,530,599,88,nn,LS82210702017269
6,548,599,91,knn,LS82210702017269
7,527,599,87,nota,LS82210702017269
8,1089,1102,98,rf,LS82210672017285
9,1086,1102,98,nn,LS82210672017285


In [7]:
col_sum = ['previsao', 'verdade', 'porcentagem', 'id_cena', 'caso']
sumario = pd.DataFrame([], columns=col_sum)
for id_cena in id_cenas:
    teste, teste_original, modelo = gera_teste_modelo(id_cena)
    
    print(dt.datetime.now(), 'inicio')
    nome_classe = 'svm'
    teste[nome_classe] = class_svm(teste, modelo, melhores_colunas)
    teste[nome_classe] = teste[nome_classe].apply(lambda x: 1 if x == 1 else 0) # coloca peso 1
    resultado = ver_resultado(teste, teste_original, nome_classe)
    resultado.append(id_cena)
    print(resultado)
    sumario = sumario.append(dict(zip(col_sum, resultado)), ignore_index=True)
    print(dt.datetime.now(), 'fim')

#     break

2018-10-30 16:18:31.159153 inicio
[805, 805, 100, 'svm', 'LS82260702017256']
2018-10-30 16:35:52.121988 fim
2018-10-30 16:35:52.373910 inicio
[525, 599, 87, 'svm', 'LS82210702017269']
2018-10-30 16:41:09.918187 fim
2018-10-30 16:41:10.162875 inicio
[1070, 1102, 97, 'svm', 'LS82210672017285']
2018-10-30 16:46:26.733555 fim
2018-10-30 16:46:26.965649 inicio
[965, 1072, 90, 'svm', 'LS82210742018128']
2018-10-30 16:51:40.010310 fim


In [15]:
sumario3=sumario.copy()

In [16]:
sumario2

,previsao,verdade,porcentagem,id_cena,caso
0,805,805,100,rf,LS82260702017256
1,799,805,99,nn,LS82260702017256
2,797,805,99,knn,LS82260702017256
3,805,805,100,nota,LS82260702017256
4,548,599,91,rf,LS82210702017269
5,531,599,88,nn,LS82210702017269
6,545,599,90,knn,LS82210702017269
7,542,599,90,nota,LS82210702017269
8,1088,1102,98,rf,LS82210672017285
9,1087,1102,98,nn,LS82210672017285


In [17]:
sumario3

,previsao,verdade,porcentagem,id_cena,caso
0,805,805,100,rf,LS82260702017256
1,802,805,99,nn,LS82260702017256
2,797,805,99,knn,LS82260702017256
3,794,805,98,nota,LS82260702017256
4,547,599,91,rf,LS82210702017269
5,530,599,88,nn,LS82210702017269
6,548,599,91,knn,LS82210702017269
7,527,599,87,nota,LS82210702017269
8,1089,1102,98,rf,LS82210672017285
9,1086,1102,98,nn,LS82210672017285
